In [40]:
import cv2
import numpy as np
import core.utils as utils
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os

In [43]:
return_elements = ["input/input_data:0", "pred_sbbox/concat_2:0", "pred_mbbox/concat_2:0", "pred_lbbox/concat_2:0"]
pb_file         = "./yolov3_coco.pb"
num_classes     = 80
input_size      = 416
graph           = tf.Graph()

In [44]:
dir = "../DroneTrainerNotebooks/data/ok"
files = [ f"{dir}/{file}" for file in os.listdir(dir) ]

In [ ]:
n = 1
original_image = cv2.imread(files[n])
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
original_image_size = original_image.shape[:2]
image_data = utils.image_preporcess(np.copy(original_image), [input_size, input_size])
image_data = image_data[np.newaxis, ...]
return_tensors = utils.read_pb_return_tensors(graph, pb_file, return_elements)

with tf.Session(graph=graph) as sess:
    pred_sbbox, pred_mbbox, pred_lbbox = sess.run(
        [return_tensors[1], return_tensors[2], return_tensors[3]],
                feed_dict={ return_tensors[0]: image_data})

pred_bbox = np.concatenate([np.reshape(pred_sbbox, (-1, 5 + num_classes)),
                            np.reshape(pred_mbbox, (-1, 5 + num_classes)),
                            np.reshape(pred_lbbox, (-1, 5 + num_classes))], axis=0)
bboxes = utils.postprocess_boxes(pred_bbox, original_image_size, input_size, 0.3)
bboxes = utils.nms(bboxes, 0.45, method='nms')

found = False
for i in range(len(bboxes)):
    if int(bboxes[i][5]) == 32:
        #print(bboxes[i][0], bboxes[i][1], bboxes[i][2], bboxes[i][3])
        found = True
        x = int(bboxes[i][0])
        y = int(bboxes[i][1])
        w = int(bboxes[i][2]) - x
        h = int(bboxes[i][3]) - y
        print(x,y,w,h)

if found:
    fig, ax = plt.subplots(1)
    rect = patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none')
    ax.imshow(original_image)
    ax.add_patch(rect)
    plt.show()
else:
    print("not found")
    plt.imshow(original_image)